# C1 W4 Group 8 - Task 1


In [1]:
from PIL import Image
from tqdm import tqdm
import cv2
import numpy as np
from src.data import AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST
from src.paths import BBDD_PATH, QSD1_W4_PATH, QSD1_NON_AUGMENTED_W4_PATH, WEEK_4_RESULTS_PATH
import matplotlib.pyplot as plt

In [2]:
BBDD_PATH_LIST = sorted(BBDD_PATH.glob("*.jpg"))
QSD1_W4_PATH_LIST = sorted(QSD1_W4_PATH.glob("*.jpg"))
QSD1_NON_AUGMENTED_W4_PATH_LIST = sorted(QSD1_NON_AUGMENTED_W4_PATH.glob("*.jpg"))

In [3]:
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in BBDD_PATH_LIST]  # Load once
for idx, db_img in enumerate(database_image_PIL_list):
    assert db_img.filename.endswith(f"{idx}.jpg")

In [4]:
query_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(query_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [5]:
non_augmented_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_NON_AUGMENTED_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(non_augmented_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [ ]:
for file, augmentation, frames, gt in zip(QSD1_W4_PATH_LIST, AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST):
    frame_info = "; ".join([f"({frame[0]:.2f}, {frame[1]})" for frame in frames])
    print(f"File={file.stem}, Aug={augmentation}, Frames={frame_info}, GT={gt}")

## Denoising over the images

In [6]:
from src.denoising import denoise_image

In [7]:
# Median filter
denoised_query_d1_PIL_list = [denoise_image(image) for image in query_d1_image_PIL_list]

In [8]:
def plot_images_with_multiple_filters(original_images, noisy_images, denoised_images, save_path):
    num_images = len(original_images)
    num_cols = 3
    fig, axes = plt.subplots(num_images, num_cols, figsize=(25, 5 * num_images))  # +1 for original image

    for i in range(num_images):
        original_image = original_images[i]
        noisy_image = noisy_images[i]

        axes[i, 0].imshow(original_image)
        axes[i, 0].set_title(f'Original Image {i+1}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(noisy_image)
        axes[i, 1].set_title(f'Noisy image Image {i+1}')
        axes[i, 1].axis('off')

        axes[i, 2].imshow(denoised_images[i])
        axes[i, 2].set_title(f'Denoised {i+1}')
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

In [ ]:
plot_images_with_multiple_filters(non_augmented_d1_image_PIL_list, query_d1_image_PIL_list,
                                   denoised_query_d1_PIL_list, WEEK_4_RESULTS_PATH / "Task_1"/ 'denoised_images_plot.png')

## Background removal

In [9]:
from src.background import get_painting_masks, crop_image_by_mask

In [10]:
def crop_paintings_from_image(image, ground_truth):
    image_np = np.array(image)
    cropped_images = []

    for points in (entry[1] for entry in ground_truth):
        src_points = np.array(points, dtype=np.float32)
        width, height = map(int, [np.linalg.norm(src_points[1] - src_points[0]), np.linalg.norm(src_points[2] - src_points[1])])
        dst_points = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

        M = cv2.getPerspectiveTransform(src_points, dst_points)
        cropped_images.append(Image.fromarray(cv2.warpPerspective(image_np, M, (width, height))))

    return cropped_images

In [11]:
cropped_query_image_list_d1 = []
pbar = tqdm(zip(sorted(QSD1_W4_PATH.glob("*.jpg")), denoised_query_d1_PIL_list, FRAMES_QSD1_W4_LIST))
for name, image, predicted_mask in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    cropped_query_image_list_d1.append(crop_paintings_from_image(image, predicted_mask))

Splitting 00029: : 30it [00:00, 31.21it/s]


In [ ]:
n_rows = len(cropped_query_image_list_d1)

fig, axes = plt.subplots(n_rows, 2, figsize=(6, n_rows *1.5))

for ax, l in zip(axes, cropped_query_image_list_d1):
    ax[0].imshow(l[0]); ax[0].axis('off')
    if len(l) > 1:
        ax[1].imshow(l[1])
    ax[1].axis('off')

plt.tight_layout()
plt.show()

## Keypoint extraction

In [12]:
from src.descriptors import SIFTDescriptor, ORBDescriptor, HOGDescriptor, ImageRetrievalSystem

In [13]:
# denoise also dataset ?
denoise_database_image = [denoise_image(image) for image in database_image_PIL_list]

In [14]:
descriptors = {
    'HOG': HOGDescriptor(),
    'SIFT': SIFTDescriptor(max_features=500),
    # 'ORB': ORBDescriptor(),
}
descriptor_path = WEEK_4_RESULTS_PATH / 'Task_1' / 'descriptors'  

retrieval_system = ImageRetrievalSystem(descriptors, descriptor_path)

In [ ]:
# Set number of top similar images to retrieve
K = 10

results = {}
g_0 = {}

for descriptor_name in descriptors.keys():
    print(f"\nRetrieving similar images using {descriptor_name} descriptor:")
    results[descriptor_name], g_0[descriptor_name] = retrieval_system.retrieve_similar_images(cropped_query_image_list_d1, denoise_database_image, K, descriptor_name, t = 0.99)


In [16]:
from src.metrics import MeanAveragePrecisionAtK

In [17]:
map = MeanAveragePrecisionAtK()

In [19]:
GT_mod = []
for element in GT_QSD1_W4_LIST:
    if len(element) == 2:
        GT_mod.extend([[element[0]], [element[1]]])
    else:
        GT_mod.append([element[0]])

results_mod = {}
for descriptor_name in descriptors.keys():
    results_mod[descriptor_name] = []
    for element in results[descriptor_name]:
        if len(element) == 2:
            results_mod[descriptor_name].extend([element[0], element[1]])
        else:
            results_mod[descriptor_name].append(element[0])

print(GT_mod,"\n"+"#" * 40)
print(f'\n{"#"*40}\n'.join([str(result) for result in results_mod.values()]))

[[-1], [150], [48], [251], [32], [161], [81], [62], [38], [-1], [128], [155], [258], [136], [76], [-1], [-1], [53], [-1], [12], [11], [280], [-1], [182], [252], [-1], [272], [117], [-1], [242], [260], [94], [132], [223], [-1], [127], [47], [13], [-1]] 
########################################
[[176, 286, 40, 193, 69, 77, 260, 67, 233, 81], [150, 184, 95, 160, 218, 232, 88, 192, 123, 244], [48, 78, 266, 111, 254, 151, 243, 221, 25, 156], [251, 11, 120, 282, 85, 32, 54, 162, 138, 180], [32, 120, 282, 39, 11, 180, 119, 138, 193, 251], [161, 40, 275, 244, 81, 286, 193, 104, 218, 88], [81, 40, 161, 193, 94, 275, 104, 67, 158, 286], [62, 115, 224, 80, 124, 53, 146, 100, 13, 107], [38, 52, 192, 160, 232, 56, 170, 2, 29, 259], [176, 193, 286, 40, 69, 77, 233, 260, 119, 81], [-1], [155, 238, 11, 139, 177, 285, 162, 138, 216, 35], [258, 144, 285, 42, 177, 246, 139, 118, 32, 218], [136, 271, 147, 82, 174, 62, 168, 218, 13, 101], [76, 120, 40, 193, 286, 69, 282, 119, 272, 233], [266, 25, 151, 200,

In [24]:
for descriptor_name, results in results_mod.items():
    print(f"Results for {descriptor_name}:")
    results = map.compute(GT_mod, results, k=K)
    print(f"{results}\n{"#" * 40}\n")

Results for HOG:
0.6923076923076923
########################################

Results for SIFT:
0.6730769230769231
########################################



In [ ]:
idx = 0
non_found_dist = []
for res, gt, gap in zip(results_mod, GT_mod, g_0):
    print(f"{idx} | Res: {res}, GT: {gt}, gaps: {gap[1]}")
    idx += 1

non_found_dist = []
for idx, (res, gt, gap) in enumerate(zip(results_mod, GT_mod, g_0)):
    gap_str = ', '.join([f"{g:.2f}" for g in gap[1]])  # Format gaps to 2 decimal places
    print(f"{idx:2} | Res: {res:<50} | GT: {gt:<10} | Gaps: [{gap_str}]")


In [ ]:
db_i = descriptors['SIFT'].compute(np.array(database_image_PIL_list[32]))
query_i = descriptors['SIFT'].compute(np.array(cropped_query_image_list_d1[4][0]))
print(db_i[0][0])

In [ ]:
from scipy.spatial.distance import cdist

distances = cdist(db_i[1], query_i[1], metric='euclidean')
min_distances = np.min(distances, axis=1)
mean = np.mean(min_distances)

print(mean)

db_i = descriptors['SIFT'].compute(np.array(database_image_PIL_list[177]))
distances = cdist(db_i[1], query_i[1], metric='euclidean')
min_distances = np.min(distances, axis=1)
mean = np.mean(min_distances)

print(mean)
